# Libraries

In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer
import re
from hazm import Normalizer
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertConfig, BertTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import hazm
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import os
import re
import json
import copy
import collections
from transformers import AutoTokenizer

# DataSet

In [1]:
!pip install gdown

In [2]:
!pip install transformers datasets torch gdown optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 45.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_6

In [3]:
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 81.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 18.7 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=4ce4b674ae9ad7b9858de232e7a96460d5b8b3fd3762264f447d9a2bb7d14d2f
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [5]:
# Downloading the dataset from the said link
import gdown

# Google Drive file ID
file_id = '1YgrCYY-Z0h2z0-PfWVfOGt1Tv0JDI-qz'
# Construct the URL to download the file
url = f'https://drive.google.com/uc?id={file_id}'
output = 'dataset.zip'

# Download the file
gdown.download(url, output, quiet=False)

# Extract the dataset
import zipfile
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('dataset')


Downloading...
From: https://drive.google.com/uc?id=1YgrCYY-Z0h2z0-PfWVfOGt1Tv0JDI-qz
To: /content/dataset.zip
100%|██████████| 13.6M/13.6M [00:00<00:00, 100MB/s] 


In [12]:
# Define the delimiter for reading CSV files
delimiter = '\t'

# Load the training dataset
train_df = pd.read_csv('/content/dataset/digimag/train.csv', delimiter=delimiter)

# Load the development dataset
dev_df = pd.read_csv('/content/dataset/digimag/dev.csv', delimiter=delimiter)

# Load the test dataset
test_df = pd.read_csv('/content/dataset/digimag/test.csv', delimiter=delimiter)

In [13]:
label_encoder = LabelEncoder() # This class is used to convert categorical labels into integer labels.
train_df['label'] = label_encoder.fit_transform(train_df['label'])
dev_df['label'] = label_encoder.transform(dev_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

# Preprocessing

In [14]:
normalizer = Normalizer()

# Function to perform additional pre-processing steps
def additional_preprocessing(text):
    # Normalize the text using Hazm normalizer
    text = normalizer.normalize(text)

    # Remove extra punctuation
    text = re.sub(r'[^\w\s]', ' ', text)

    # Transform Arabic characters to Persian ones
    arabic_to_persian = {
        'ي': 'ی', 'ك': 'ک', 'ؤ': 'و', 'ى': 'ی', 'ة': 'ه', 'ۀ': 'ه'
    }
    for arabic_char, persian_char in arabic_to_persian.items():
        text = text.replace(arabic_char, persian_char)

    # Remove repeated letters more than twice in non-standard Persian words
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # Remove Arabic diacritics
    text = re.sub(r'[\u064B-\u065F\u0670]', '', text)

    # Replace non-Persian characters with space
    text = re.sub(r'[^آ-ی\s]', ' ', text)

    # Remove hashtag sign while preserving hashtag information
    text = re.sub(r'#(\w+)', r'\1', text)

    # Replace Persian numeric characters with English numeric characters
    persian_to_english_numbers = {
        '۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4',
        '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'
    }
    for persian_num, english_num in persian_to_english_numbers.items():
        text = text.replace(persian_num, english_num)

    # Remove weird characters and emojis
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)

    # Removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

In [15]:
# Load the tokenizer for AriaBERT model
tokenizer = AutoTokenizer.from_pretrained('ViraIntelligentDataMining/AriaBERT')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Visualization

In [16]:
# Print dataset information
print('Data Information: ')
print(train_df.info(), '\n')

Data Information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6896 entries, 0 to 6895
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6896 non-null   int64 
 1   content     6896 non-null   object
 2   label       6896 non-null   int64 
 3   label_id    6896 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 215.6+ KB
None 



In [17]:
# Function to apply additional preprocessing to a dataframe
def preprocess_dataframe(df):
    # Apply additional_preprocessing function to the 'content' column and store the result in 'text' column
    df['text'] = df['content'].apply(additional_preprocessing)

    # Return a new dataframe with only the 'text' and 'label' columns
    return df[['text', 'label']]

# Apply preprocessing to the training dataframe
train_df = preprocess_dataframe(train_df)

# Apply preprocessing to the development dataframe
dev_df = preprocess_dataframe(dev_df)

# Apply preprocessing to the test dataframe
test_df = preprocess_dataframe(test_df)

# Get the count of data samples per label in the training dataframe
label_counts = train_df['label'].value_counts()

# Display the count of data samples per label
print("Count of data samples per label is:")
print(label_counts)

Count of data samples per label is:
label
3    2245
0    1593
5    1350
2    1304
6     206
1     101
4      97
Name: count, dtype: int64


# Class Mapping

In [18]:
class_mapping = {4: 6, 1: 6}

# Update labels in the dataset based on the class mapping
train_df['label'] = train_df['label'].replace(class_mapping)
dev_df['label'] = dev_df['label'].replace(class_mapping)
test_df['label'] = test_df['label'].replace(class_mapping)

# Get the count of data samples per updated label
updated_label_counts = train_df['label'].value_counts()

# Display the count of data samples per updated label
print("Count of data samples per updated label is:")
print(updated_label_counts)

Count of data samples per updated label is:
label
3    2245
0    1593
5    1350
2    1304
6     404
Name: count, dtype: int64


In [19]:
# Convert dataframes to datasets
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

In [20]:
# Defining a function for tokenizing the text data
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

# Apply tokenization to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
dev_dataset = dev_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6896 [00:00<?, ? examples/s]

Map:   0%|          | 0/767 [00:00<?, ? examples/s]

Map:   0%|          | 0/852 [00:00<?, ? examples/s]

# Cuda

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

device: cuda:0


# Model

General configuration variables

In [22]:
MAX_LEN = 128  # Maximum length of input sequences
TRAIN_BATCH_SIZE = 16  # Batch size for training
VALID_BATCH_SIZE = 16  # Batch size for validation
TEST_BATCH_SIZE = 16  # Batch size for testing

EPOCHS = 3  # Number of epochs for training
EVERY_EPOCH = 1000  # Frequency of printing training progress
LEARNING_RATE = 2e-5  # Learning rate for optimization
CLIP = 0.0  # Gradient clipping threshold

MODEL_NAME_OR_PATH = 'ViraIntelligentDataMining/AriaBERT'  # Pretrained model name or path
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'  # Output path for the trained model

# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [23]:
print("Label encoder classes are:", label_encoder.classes_)

Label encoder classes are: ['بازی ویدیویی' 'راهنمای خرید' 'سلامت و زیبایی' 'علم و تکنولوژی' 'عمومی'
 'هنر و سینما' 'کتاب و ادبیات']


Labeling


In [27]:
id_to_label = {
    0: "عمومی",
    6: "بقیه",
    2: "سلامت و زیبایی",
    3: "راهنمای خرید",
    5: "کتاب و ادبیات",
}
label_to_id = {
     "عمومی" :0,
     "بقیه" :6,
     "سلامت و زیبایی" : 2,
     "راهنمای خرید" : 3,
     "کتاب و ادبیات" : 5,
}
id_to_label = {label: id for id, label in id_to_label.items()}
labels = [item for item in label_to_id.keys()]
labels.reverse()

In [26]:
print("these are the modified labels: ",labels)

these are the modified labels:  ['کتاب و ادبیات', 'راهنمای خرید', 'سلامت و زیبایی', 'بقیه', 'عمومی']
